## Plot pLDDT vs Pocket Residue Number

In [1]:
import os
import pandas as pd
import numpy as np

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import PDBIO
from Bio.PDB.PDBIO import Select
from Bio.Seq import Seq 

import plotly.express as px
import plotly.graph_objs as go


### Read inputs

In [2]:
metadata=pd.read_csv('inputs/alphafold_predictions_sample.csv') #contains list of AF-predicted kinase structures
pocket_dir='inputs/AF_pockets' #directory containing PDB files of AF2 structures (only pocket residues extracted) 

### Method to draw shaded region (error bands) 

In [3]:
def line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands."""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            #draw error bar (mean +/- standard deviation)
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data (adapted from https://stackoverflow.com/a/66854398/8849755)
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

### Method to create plddt dataframe

In [4]:
def create_plddt_df(metadata):
    plddt_df = pd.DataFrame([],columns=['uniprot','pocket','plddt'])
    pdbparser = PDBParser() 
    count=0
    for idx,row in metadata.iterrows():
        pdb_id = row.pdb_id
        pdb_path='/'.join([pocket_dir,pdb_id+'_pocket.pdb'])
        resid_path='/'.join([pocket_dir,pdb_id+'_pocket_idx.txt'])
        uniprot = pdb_id.split('-')[1]

        ##get bfactors of KLIFS pocket residues
        try:
            count = count+1
            with open(pdb_path, "r") as f:
                structure = pdbparser.get_structure('0', pdb_path) #Extract structure
            with open(resid_path, "r") as g:
                pocket = g.readlines()[0].split(' ') #pocket residue ID list
                if len(pocket) < 85:
                    raise
        except:
            continue

        bfactors=[]
        residues = structure.get_residues()
        #extract bfactor from structure
        for i in range(85):
            if '-' in pocket[i]:
                bfactors.append('-')
            else:
                residue = next(residues)
                atom = next(residue.get_atoms())
                bfactors.append(atom.get_bfactor())

        uniprot = [uniprot] * 85
        pocket = list(range(1,86))
        
        #append to dataframe
        df = pd.DataFrame({'uniprot':uniprot,'pocket':pocket,'plddt':bfactors})
        plddt_df = pd.concat([plddt_df,df],ignore_index=True)
    
    plddt_df = plddt_df.replace('-', np.nan)
    
    #compute mean, min, max plddt
    plot_plddt = plddt_df[['pocket','plddt']]
    plot_plddt = plot_plddt.groupby('pocket').mean()
    plot_plddt['min'] = plddt_df[['pocket','plddt']].groupby('pocket').min()
    plot_plddt['max'] = plddt_df[['pocket','plddt']].groupby('pocket').max()
    plot_plddt['std'] = plddt_df[['pocket','plddt']].groupby('pocket').std()
    plot_plddt = plot_plddt.reset_index()
        
    return plot_plddt

## Plot pLDDT vs Pocket Residue Index

In [5]:
plot_plddt = create_plddt_df(metadata)

fig = line(
    data_frame = plot_plddt,
    x = 'pocket',
    y = 'plddt',
    error_y = 'std',
    error_y_mode = 'band',
    markers = '.'
    )
fig.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        dtick =1
    )
)

fig.update_xaxes(range=[0, 85])
fig.update_yaxes(range=[55, 100])
fig.write_image("outputs/plddt.png",scale=6, width=1080)
fig.show()